In [1]:
from pandas.io.pytables import AppendableFrameTable
from requests.api import get
from wrapper.kiteext import KiteExt
from utils import Login
from utils import SupportResistance
from utils import Telegrambot
from utils import CandlePattern
from instruments import Instruments
import json
import pandas as pd
import logging
from time import sleep
from datetime import datetime as dt
from datetime import timedelta as td
import schedule
import numpy as np
from utils import Utils
from ticker import ZerodhaTicker
import time
from multiprocessing import Process
import talib


LoginWrapper = Login.Login()
kite = LoginWrapper.loginkite()
SupportResistanceWrapper = SupportResistance.SupportResistance()
CandlePatternWrapper = CandlePattern.CandlePattern()
TelegrambotWrapper = Telegrambot.Telegrambot()
Instruments = Instruments.Instruments(kite)
Utils = Utils.Utils()

In [5]:
fnostoks = ['AARTIIND','ABFRL','ACC','ADANIENT','ADANIPORTS','ALKEM','AMARAJABAT','AMBUJACEM','APLLTD','APOLLOHOSP',
            'APOLLOTYRE','ASHOKLEY','ASIANPAINT','ASTRAL','AUBANK','AUROPHARMA','AXISBANK','BAJAJ-AUTO','BAJAJFINSV',
            'BAJFINANCE','BALKRISIND','BANDHANBNK','BANKBARODA','BATAINDIA','BEL','BERGEPAINT','BHARATFORG','BHARTIARTL',
            'BHEL','BIOCON','BOSCHLTD','BPCL','BRITANNIA','CADILAHC','CANBK','CANFINHOME','CHOLAFIN','CIPLA','COALINDIA',
            'COFORGE','COLPAL','CONCOR','COROMANDEL','CUB','CUMMINSIND','DABUR','DEEPAKNTR','DIVISLAB','DIXON','DLF',
            'DRREDDY','EICHERMOT','ESCORTS','EXIDEIND','FEDERALBNK','GAIL','GLENMARK','GMRINFRA','GODREJCP','GODREJPROP',
            'GRANULES','GRASIM','GUJGASLTD','HAL','HAVELLS','HCLTECH','HDFC','HDFCAMC','HDFCBANK','HDFCLIFE','HEROMOTOCO',
            'HINDALCO','HINDPETRO','HINDUNILVR','IBULHSGFIN','ICICIBANK','ICICIGI','ICICIPRULI','IDEA','IDFCFIRSTB',
            'IEX','IGL','INDHOTEL','INDIAMART','INDIGO','INDUSINDBK','INDUSINDBK','INFY','IOC','IPCALAB','IRCTC','ITC',
            'JINDALSTEL','JSWSTEEL','JUBLFOOD','KOTAKBANK','L&TFH','LALPATHLAB','LICHSGFIN','LT','LTI','LTTS','LUPIN','M&M',
            'M&MFIN','MANAPPURAM','MARICO','MARUTI','MCDOWELL-N','MCX','METROPOLIS','MFSL','MGL','MINDTREE','MOTHERSUMI',
            'MPHASIS','MRF','MUTHOOTFIN','NAM-INDIA','NATIONALUM','NAUKRI','NAVINFLUOR','NESTLEIND','NMDC','NTPC',
            'OFSS','ONGC','PAGEIND','PEL','PETRONET','PFC','PFIZER','PIDILITIND','PIIND','PNB','POLYCAB','POWERGRID',
            'PVR','RAMCOCEM','RBLBANK','RECLTD','RELIANCE','SAIL','SBILIFE','SBIN','SHREECEM','SIEMENS','SRF','SRTRANSFIN',
            'STAR','SUNPHARMA','SUNTV','SYNGENE','TATACHEM','TATACONSUM','TATAMOTORS','TATAPOWER','TATASTEEL','TCS','TECHM',
            'TITAN','TORNTPHARM','TORNTPOWER','TRENT','TVSMOTOR','UBL','ULTRACEMCO','UPL','VEDL','VOLTAS','WIPRO','ZEEL']

In [4]:
test = kite.positions()

In [8]:
test1 = pd.DataFrame(test['net'])

In [11]:
test1

,tradingsymbol,exchange,instrument_token,product,quantity,overnight_quantity,multiplier,average_price,close_price,last_price,...,sell_quantity,sell_price,sell_value,sell_m2m,day_buy_quantity,day_buy_price,day_buy_value,day_sell_quantity,day_sell_price,day_sell_value
0,BANKNIFTY2191636600CE,NFO,11784962,NRML,0,0,1,0,0,192.30,...,50,209.40,10470.00,10470.00,50,204.025,10201.25,50,209.40,10470.00
1,BANKNIFTY2191636600PE,NFO,11785474,NRML,0,0,1,0,0,161.35,...,25,161.45,4036.25,4036.25,25,175.000,4375.00,25,161.45,4036.25
2,BANKNIFTY2191636800CE,NFO,11786498,NRML,0,0,1,0,0,103.00,...,50,150.40,7520.00,7520.00,50,154.525,7726.25,50,150.40,7520.00
3,BANKNIFTY2191636800PE,NFO,11788034,NRML,0,0,1,0,0,271.00,...,25,259.00,6475.00,6475.00,25,250.650,6266.25,25,259.00,6475.00


In [20]:
above_rsi = []
below_ris = []

In [41]:
data = pd.DataFrame()
for scrips in fnostoks:
    token = Instruments.getInstrumentDataBySymbol(scrips,'NSE')
    df = pd.DataFrame(kite.historical_data(token, dt.today() - td(days=90), dt.today(), 'day'))
    df['stock'] = scrips
    data = data.append(df)
    real = talib.RSI(df['close'], timeperiod=14)
    if real.iloc[-1] > 80:
        above_rsi.append(scrips)
    elif real.iloc[-1] < 30:
        below_ris.append(scrips)
    sleep(5)

In [58]:
df = data[data['stock'] == 'MARICO']

In [84]:
last_2_smas_higher = []
for scrips in fnostoks:
    df = data[data['stock'] == scrips]
    smas = pd.DataFrame()
    smas['sma_20'] = talib.SMA(df['close'],20)
    smas['sma_50'] = talib.SMA(df['close'],50)
    smas['greater'] = smas['sma_20'] > smas['sma_50']
    if (smas.iloc[-2]['greater'] == True) & (smas.iloc[-1]['greater'] == True):
        if (smas.iloc[-5]['greater'] == False) | (smas.iloc[-4]['greater'] == False):
            last_2_smas_higher.append(scrips)
    
    

In [85]:
last_2_smas_higher


['BEL',
 'CANFINHOME',
 'CONCOR',
 'GAIL',
 'ICICIGI',
 'IOC',
 'MARICO',
 'PIDILITIND']

In [60]:
sma_20 = talib.SMA(df['close'],20)
sma_50 = talib.SMA(df['close'],50)

In [61]:
smas = pd.DataFrame()
smas['sma_20'] = sma_20
smas['sma_50'] = sma_50 

In [62]:
smas['greater'] = smas['sma_20'] > smas['sma_50']

In [63]:
smas['greater']

0     False
1     False
2     False
3     False
4     False
      ...  
56    False
57    False
58     True
59     True
60     True
Name: greater, Length: 61, dtype: bool

In [66]:
(smas.iloc[-2]['greater'] == True) & (smas.iloc[-1]['greater'] == True)

True